In [5]:
from collections import namedtuple
Vertex = namedtuple('Vertex', ('tails', 'heads'))

def graph(filename):
    with open(filename) as f:
        print('{} vertices, {} edges'.format(*f.readline()[:-1].split()))
        graph, neg = dict(), False
        for l in f:
            u, v, c = tuple(int(x) for x in l.split())
            if u not in graph:
                graph[u] = Vertex(dict(), dict())
            if v not in graph:
                graph[v] = Vertex(dict(), dict())
            graph[u].heads[v] = c
            graph[v].tails[u] = c
            if c < 0:
                neg = True
    return graph, neg

In [17]:
def floyd_warshall(g):
    vertexes = tuple(g.keys())
    a = [{v: {w: 0 if w == v else (g[v].heads[w] if w in g[v].heads else float('inf')) for w in vertexes} for v in vertexes},
         {v: {w: 0 if w == v else (g[v].heads[w] if w in g[v].heads else float('inf')) for w in vertexes} for v in vertexes}]
    for i in range(1, len(vertexes)):
        ii = i % 2
        curcol, prevcol = a[ii], a[ii-1]
        for u in vertexes:
            for v in vertexes:
                curcol[u][v] = min(prevcol[u][v], prevcol[u][i] + prevcol[i][v])
    return curcol

In [3]:
def bellman_ford(g, s):
    vertexes = tuple(g.keys())
    a = [{v: 0 if v == s else float('inf') for v in vertexes}, dict()]
    for i in range(1, len(vertexes) + 1):
        ii = i % 2
        curcol, prevcol = a[ii], a[ii-1]
        for v in vertexes:
            try:
                curcol[v] = min(prevcol[v], min(prevcol[x] + g[v].tails[x] for x in g[v].tails))
            except ValueError: # the min() was empty
                curcol[v] = prevcol[v]
    return a

In [26]:
from heapq import heappush, heappop

def dijkstra(g, s):
    paths = dict()
    adj_v = list() # heap
    paths[s] = 0
    for v in g[s].heads:
        heappush(adj_v, (g[s].heads[v], v))
    while len(adj_v) > 0:
        greedy, u = heappop(adj_v)
        if u not in paths:
            paths[u] = greedy
            for v in g[u].heads:
                if v not in paths:
                    heappush(adj_v, (g[u].heads[v], v))
    return paths

In [34]:
def johnson(g):
    print("Running Johnson")
    # 1. Creating a copy of the input graph
    print("Copying the input graph")
    gg = {v: Vertex(g[v].tails.copy(), g[v].heads.copy()) for v in g}
    # 2. Add node (-1) as a tail to all other nodes, with weight 0
    print("1 Reweighting: Adding virtual node and connecting it to all other nodes")
    for v in gg:
        gg[v].tails[-1] = 0
    gg[-1] = Vertex(tuple(), tuple())
    # 3. Run BF with source -1
    print("1 Reweighting: Running Bellman-Ford to compute node labels")
    a = bellman_ford(gg, -1)
    # Found negative cycle?
    if a[0] != a[1]:
        raise ValueError('input graph contains a negative cycle!')
    # 4. Create another copy with new edge weights
    print("1 Reweighting: computing new edge weights")
    gg = {v: Vertex(tuple(), {w: g[v].heads[w] + a[0][v] - a[0][w] for w in g[v].heads}) for v in g}
    # 5. Running Dijkstra for every source - destination pair
    print("2 Running Dijkstra {} times".format(len(g)))
    result = dict()
    for v in gg:
        d = dijkstra(gg, v)
        result[v] = {w: d[w] - a[0][v] + a[0][w] for w in d}
    return result

In [35]:
#for filename in ('g1.txt', 'g2.txt', 'g3.txt'):
for filename in ('g3.txt',):
    print("Computing shortest paths in {}".format(filename))
    g = graph(filename)
    try:
        a = johnson(g[0])
    except ValueError as e:
        print(e)
    else:
        print(min(a[x][y] for x in a for y in a[x]))

Computing shortest paths in g3.txt
1000 vertices, 47978 edges
Running Johnson
Copying the input graph
1 Reweighting: Adding virtual node and connecting it to all other nodes
1 Reweighting: Running Bellman-Ford to compute node labels
1 Reweighting: computing new edge weights
2 Running Dijkstra 1000 times
-19
